In [1]:
import re
import numpy as np
import pandas as pd

from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score, GridSearchCV, ShuffleSplit

%matplotlib inline

# Import Data

In [2]:
data = pd.read_csv('./data/train.csv')
data.drop('id', axis = 1, inplace = True)
X_train = np.asarray(data[data.columns[range(1, data.shape[1])]], dtype = np.double)
Y_train = np.asarray(data[['label']], dtype = np.double).ravel()
data.head()

,label,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,...,feat207,feat208,feat209,feat210,feat211,feat212,feat213,feat214,feat215,feat216
0,1,7,0,3,0,2,3,0,6,0,...,3,4,2,2,0,13,0,11,1,3
1,1,0,11,0,0,10,1,0,0,4,...,0,2,0,0,2,8,1,13,0,4
2,0,9,0,3,0,1,3,0,4,0,...,48,11,2,0,0,4,0,2,0,0
3,0,0,9,3,2,25,0,4,0,0,...,1,14,1,0,0,0,3,0,17,1
4,0,0,0,0,0,2,5,0,0,0,...,3,12,0,3,0,4,0,24,4,0


In [3]:
data.groupby(['label']).count()

,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,...,feat207,feat208,feat209,feat210,feat211,feat212,feat213,feat214,feat215,feat216
label,,,,,,,,,,,,,,,,,,,,,
0,327,327,327,327,327,327,327,327,327,327,...,327,327,327,327,327,327,327,327,327,327
1,573,573,573,573,573,573,573,573,573,573,...,573,573,573,573,573,573,573,573,573,573


# Fit Methods

#Подберем параметры для svc

In [4]:
param_grid = {'C':[1,10,100,1000],'gamma':[1,0.1,0.001,0.0001], 'kernel':['linear','rbf']}

In [5]:
grid = GridSearchCV(SVC(),param_grid,refit = True, verbose=2)

In [6]:
%time grid.fit(X_train, Y_train)

Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV] C=1, gamma=1, kernel=linear .....................................
[CV] ...................... C=1, gamma=1, kernel=linear, total=   0.5s
[CV] C=1, gamma=1, kernel=linear .....................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV] ...................... C=1, gamma=1, kernel=linear, total=   0.3s
[CV] C=1, gamma=1, kernel=linear .....................................
[CV] ...................... C=1, gamma=1, kernel=linear, total=   0.5s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] ......................... C=1, gamma=1, kernel=rbf, total=   0.5s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] ......................... C=1, gamma=1, kernel=rbf, total=   0.4s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV] ......................... C=1, gamma=1, kernel=rbf, total=   0.6s
[CV] C=1, gamma=0.1, kernel=linear ...................................
[CV] .................... C=1, gamma=0.1, kernel=linear, total=   0.7s
[CV] C=1, gamma=0.1, kernel=linear ...................................
[CV] .................... C=1, gamma=0.1, kernel=linear, total=   0.3s
[CV] C=1, gamma=0.1, kernel=linear ...................................
[CV] .

[CV] ..................... C=100, gamma=0.1, kernel=rbf, total=   0.5s
[CV] C=100, gamma=0.001, kernel=linear ...............................
[CV] ................ C=100, gamma=0.001, kernel=linear, total=   0.5s
[CV] C=100, gamma=0.001, kernel=linear ...............................
[CV] ................ C=100, gamma=0.001, kernel=linear, total=   0.5s
[CV] C=100, gamma=0.001, kernel=linear ...............................
[CV] ................ C=100, gamma=0.001, kernel=linear, total=   0.5s
[CV] C=100, gamma=0.001, kernel=rbf ..................................
[CV] ................... C=100, gamma=0.001, kernel=rbf, total=   0.3s
[CV] C=100, gamma=0.001, kernel=rbf ..................................
[CV] ................... C=100, gamma=0.001, kernel=rbf, total=   0.3s
[CV] C=100, gamma=0.001, kernel=rbf ..................................
[CV] ................... C=100, gamma=0.001, kernel=rbf, total=   0.4s
[CV] C=100, gamma=0.0001, kernel=linear ..............................
[CV] .

[Parallel(n_jobs=1)]: Done  96 out of  96 | elapsed:  1.0min finished


Wall time: 1min 3s


GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [1, 10, 100, 1000], 'gamma': [1, 0.1, 0.001, 0.0001], 'kernel': ['linear', 'rbf']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [7]:
grid.best_params_

{'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}

In [8]:
#попробуем попдобрать параметры немного лучше, на основе предыдущей попытки
param_grid = {'C':[1,5,7, 8.5, 10],'gamma':np.linspace(0.001,0.0001, 100), 'kernel':['linear','rbf']}
grid = GridSearchCV(SVC(),param_grid,refit = True, verbose=2)
%time grid.fit(X_train, Y_train)

Fitting 3 folds for each of 1000 candidates, totalling 3000 fits
[CV] C=1, gamma=0.001, kernel=linear .................................
[CV] .................. C=1, gamma=0.001, kernel=linear, total=   0.5s
[CV] C=1, gamma=0.001, kernel=linear .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV] .................. C=1, gamma=0.001, kernel=linear, total=   0.4s
[CV] C=1, gamma=0.001, kernel=linear .................................
[CV] .................. C=1, gamma=0.001, kernel=linear, total=   0.5s
[CV] C=1, gamma=0.001, kernel=rbf ....................................
[CV] ..................... C=1, gamma=0.001, kernel=rbf, total=   0.4s
[CV] C=1, gamma=0.001, kernel=rbf ....................................
[CV] ..................... C=1, gamma=0.001, kernel=rbf, total=   0.3s
[CV] C=1, gamma=0.001, kernel=rbf ....................................
[CV] ..................... C=1, gamma=0.001, kernel=rbf, total=   0.4s
[CV] C=1, gamma=0.000990909090909091, kernel=linear ..................
[CV] ... C=1, gamma=0.000990909090909091, kernel=linear, total=   0.5s
[CV] C=1, gamma=0.000990909090909091, kernel=linear ..................
[CV] ... C=1, gamma=0.000990909090909091, kernel=linear, total=   0.3s
[CV] C=1, gamma=0.000990909090909091, kernel=linear ..................
[CV] .

[CV] ..... C=1, gamma=0.0009181818181818182, kernel=rbf, total=   0.3s
[CV] C=1, gamma=0.0009090909090909091, kernel=linear .................
[CV] .. C=1, gamma=0.0009090909090909091, kernel=linear, total=   0.4s
[CV] C=1, gamma=0.0009090909090909091, kernel=linear .................
[CV] .. C=1, gamma=0.0009090909090909091, kernel=linear, total=   0.3s
[CV] C=1, gamma=0.0009090909090909091, kernel=linear .................
[CV] .. C=1, gamma=0.0009090909090909091, kernel=linear, total=   0.4s
[CV] C=1, gamma=0.0009090909090909091, kernel=rbf ....................
[CV] ..... C=1, gamma=0.0009090909090909091, kernel=rbf, total=   0.3s
[CV] C=1, gamma=0.0009090909090909091, kernel=rbf ....................
[CV] ..... C=1, gamma=0.0009090909090909091, kernel=rbf, total=   0.3s
[CV] C=1, gamma=0.0009090909090909091, kernel=rbf ....................
[CV] ..... C=1, gamma=0.0009090909090909091, kernel=rbf, total=   0.4s
[CV] C=1, gamma=0.0009, kernel=linear ................................
[CV] .

[CV] ..... C=1, gamma=0.0008272727272727273, kernel=rbf, total=   0.3s
[CV] C=1, gamma=0.0008272727272727273, kernel=rbf ....................
[CV] ..... C=1, gamma=0.0008272727272727273, kernel=rbf, total=   0.3s
[CV] C=1, gamma=0.0008272727272727273, kernel=rbf ....................
[CV] ..... C=1, gamma=0.0008272727272727273, kernel=rbf, total=   0.3s
[CV] C=1, gamma=0.0008181818181818182, kernel=linear .................
[CV] .. C=1, gamma=0.0008181818181818182, kernel=linear, total=   0.4s
[CV] C=1, gamma=0.0008181818181818182, kernel=linear .................
[CV] .. C=1, gamma=0.0008181818181818182, kernel=linear, total=   0.3s
[CV] C=1, gamma=0.0008181818181818182, kernel=linear .................
[CV] .. C=1, gamma=0.0008181818181818182, kernel=linear, total=   0.4s
[CV] C=1, gamma=0.0008181818181818182, kernel=rbf ....................
[CV] ..... C=1, gamma=0.0008181818181818182, kernel=rbf, total=   0.3s
[CV] C=1, gamma=0.0008181818181818182, kernel=rbf ....................
[CV] .

...

[CV] ... C=10, gamma=0.00024545454545454545, kernel=rbf, total=   0.3s
[CV] C=10, gamma=0.00024545454545454545, kernel=rbf ..................
[CV] ... C=10, gamma=0.00024545454545454545, kernel=rbf, total=   0.2s
[CV] C=10, gamma=0.00024545454545454545, kernel=rbf ..................
[CV] ... C=10, gamma=0.00024545454545454545, kernel=rbf, total=   0.2s
[CV] C=10, gamma=0.00023636363636363633, kernel=linear ...............
[CV]  C=10, gamma=0.00023636363636363633, kernel=linear, total=   0.5s
[CV] C=10, gamma=0.00023636363636363633, kernel=linear ...............
[CV]  C=10, gamma=0.00023636363636363633, kernel=linear, total=   0.3s
[CV] C=10, gamma=0.00023636363636363633, kernel=linear ...............
[CV]  C=10, gamma=0.00023636363636363633, kernel=linear, total=   0.4s
[CV] C=10, gamma=0.00023636363636363633, kernel=rbf ..................
[CV] ... C=10, gamma=0.00023636363636363633, kernel=rbf, total=   0.3s
[CV] C=10, gamma=0.00023636363636363633, kernel=rbf ..................
[CV] .

[CV]  C=10, gamma=0.00015454545454545452, kernel=linear, total=   0.3s
[CV] C=10, gamma=0.00015454545454545452, kernel=linear ...............
[CV]  C=10, gamma=0.00015454545454545452, kernel=linear, total=   0.4s
[CV] C=10, gamma=0.00015454545454545452, kernel=rbf ..................
[CV] ... C=10, gamma=0.00015454545454545452, kernel=rbf, total=   0.1s
[CV] C=10, gamma=0.00015454545454545452, kernel=rbf ..................
[CV] ... C=10, gamma=0.00015454545454545452, kernel=rbf, total=   0.1s
[CV] C=10, gamma=0.00015454545454545452, kernel=rbf ..................
[CV] ... C=10, gamma=0.00015454545454545452, kernel=rbf, total=   0.1s
[CV] C=10, gamma=0.0001454545454545455, kernel=linear ................
[CV] . C=10, gamma=0.0001454545454545455, kernel=linear, total=   0.4s
[CV] C=10, gamma=0.0001454545454545455, kernel=linear ................
[CV] . C=10, gamma=0.0001454545454545455, kernel=linear, total=   0.3s
[CV] C=10, gamma=0.0001454545454545455, kernel=linear ................
[CV] .

[Parallel(n_jobs=1)]: Done 3000 out of 3000 | elapsed: 27.9min finished


Wall time: 27min 56s


GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [1, 5, 7, 8.5, 10], 'gamma': array([0.001  , 0.00099, ..., 0.00011, 0.0001 ]), 'kernel': ['linear', 'rbf']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [14]:
grid.best_params_, grid.best_score_

({'C': 5, 'gamma': 0.0006272727272727273, 'kernel': 'rbf'}, 0.9155555555555556)

# Get final classifier

In [ ]:
#params = {
#    'penalty':['l1', 'l2'],        # l1 is Lasso, l2 is Ridge
#    'solver':['liblinear'],
#    'C': np.concatenate((np.linspace(.1, 1., 17), np.linspace(10, 200, 20)))
#}

In [ ]:
#lr_grid = {'C': np.concatenate((np.linspace(.1, .9, 9), np.linspace(10, 200, 20)))}
#gs = GridSearchCV(LogisticRegression(), params, scoring='accuracy', cv=5).fit(X_train, Y_train)

#lr = LogisticRegression()
#%time lr_gs = GridSearchCV(lr, params, cv=3, scoring='roc_auc').fit(X_train, Y_train)
#%time  gs.fit(X_train, Y_train)
#lr_gs.best_params_, lr_gs.best_score_

In [49]:
#lr_grid = {'C': np.concatenate((np.linspace(.1, .9, 9), np.linspace(10, 200, 20)))}
#gs = GridSearchCV(LogisticRegression(), params, scoring='accuracy', cv=5).fit(X_train, Y_train)

#lr = LogisticRegression()
#%time lr_gs = GridSearchCV(lr, params, cv=5, scoring='roc_auc').fit(X_train, Y_train)
#%time  gs.fit(X_train, Y_train)
#lr_gs.best_params_, lr_gs.best_score_

Wall time: 3min 58s


({'C': 0.49375, 'penalty': 'l1', 'solver': 'liblinear'}, 0.9330490541749122)

In [42]:
#estimator = LogisticRegression() #benchmark_3
#estimator.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [50]:
#estimator = LogisticRegression(C=0.49375, penalty='l1') #benchmark_4
#estimator.fit(X_train, Y_train)

LogisticRegression(C=0.49375, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

Попробуем подобрать параметры для SVC

In [55]:
#Пробный запуск
#estimator =  SVC()#benchmark_5 0.78482
#estimator.fit(X_train, Y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [9]:
#После подбора параметров
#estimator =  SVC(C=10, gamma=0.0001, kernel='rbf')#benchmark_6 0.91004
#estimator.fit(X_train, Y_train)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [10]:
#После улучшенного подбора параметров
estimator =  SVC(C=5, gamma=0.0006272727272727273, kernel='rbf')#benchmark_7  0.92049
estimator.fit(X_train, Y_train)

SVC(C=5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0006272727272727273,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

# Test data estimation

In [11]:
test = pd.read_csv('./data/test.csv')
X_test = np.asarray(test[test.columns[range(1, test.shape[1])]])

In [12]:
X_test

array([[ 4,  2,  2, ...,  6,  2,  3],
       [ 3,  3,  0, ..., 27,  2,  0],
       [ 0,  0,  0, ...,  5,  1,  1],
       ...,
       [ 1,  1,  0, ..., 36,  0,  2],
       [ 1,  3,  2, ..., 35,  1,  0],
       [ 0,  2,  3, ..., 15,  0,  0]], dtype=int64)

In [13]:
test_predict=estimator.predict(X_test)
test['label'] = test_predict
test[['id', 'label']].to_csv('benchmark_7.csv', sep = ',', index = False)